# Lab 1-1: Train Hugging Face Transformers on Local Environment

### Multi-Class Classification with Naver Movie dataset and Hugging Face `Trainer` 
---

## Introduction
---

본 모듈에서는 Hugging Face `transformers` 및 `datasets` 라이브러리를 사용하여 한국어 텍스트 감성 분류 파인 튜닝을 수행합니다. 파인 튜닝한 모델은 SageMaker 상에서 실시간 엔드포인트(real-time endpoint)/비동기 엔드포인트(asynchronous endpoint)/배치 변환(batch transform)/서버리스 엔드포인트(serverless endpoint)의 다양한 형태로 배포할 수 있으며, 모델 아티팩트를 Hugging Face Hub에 등록하여 아래와 같이 추론 결과를 웹으로 확인할 수도 있습니다.

Reference: https://huggingface.co/docs/transformers/training

_**Note: SageMaker Studio Lab, SageMaker Studio, SageMaker 노트북 인스턴스, 또는 여러분의 로컬 머신에서 이 데모를 실행할 수 있습니다. SageMaker Studio Lab을 사용하는 경우 GPU를 활성화하세요.**_

In [ ]:
import os

try:
    import torch
except ImportError:
    os.system('pip install torch==1.8.1')

<div class="alert alert-warning"><h4>주의</h4><p>
    
이 예제 노트북은 **transformers v4.11.0** 이상이 필요합니다.    
아래 코드 셀은 핸즈온에 필요한 라이브러리들을 설치하고, 주피터 노트북 커널을 셧다운시킵니다. 
    
노트북 커널이 셧다운된다면, 아래 코드 셀에서 <b><font color='darkred'>install_needed = False</font></b>로 변경 후, 코드 셀을 다시 실행해 주세요. 이 작업은 한 번만 수행하면 됩니다. 
</p></div>

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import IPython

#install_needed = True
install_needed = False

if install_needed:
    print("===> Installing deps and restarting kernel. Please change 'install_needed = False' and run this code cell again.")
    !{sys.executable} -m pip install -U transformers s3fs datasets
    IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
import os
import json
import sys
import logging
import argparse
from datasets import load_dataset
from transformers import (
    ElectraModel, ElectraTokenizer, ElectraForSequenceClassification, Trainer, TrainingArguments, set_seed
)
from transformers.trainer_utils import get_last_checkpoint

logging.basicConfig(
    level=logging.INFO, 
    format='[{%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

<br>

## 1. Preparation
---

빠른 핸즈온을 위해 네이버 영화 리뷰 말뭉치 데이터셋으로 사전에 파인튜닝된 모델을 그대로 사용합니다. (즉, 파인튜닝 없이도 이미 해당 데이터셋에 대해 정확도; 약 89% accuracy 를 보입니다) 처음부터 파인튜닝을 진행하고 싶다면 주석을 해제해 주세요.

In [ ]:
# Define the model repo
tokenizer_id = 'daekeun-ml/koelectra-small-v3-nsmc'
model_id = "daekeun-ml/koelectra-small-v3-nsmc"
# tokenizer_id = 'monologg/koelectra-small-v3-discriminator'
# model_id = "monologg/koelectra-small-v3-discriminator"

# dataset used
dataset_name = 'nsmc'

### Dataset

본 핸즈온에서 사용할 말뭉치 데이터셋은 네이버 영화 리뷰 감성 분류 데이터(https://github.com/e9t/nsmc/) 공개 데이터셋으로 15만 건의 훈련 데이터와 5만 건의 테스트 데이터로 구성되어 있습니다. 이 데이터셋은 한국어 자연어 처리 모델 벤치마킹에 자주 사용됩니다.

![emotion-widget.png](../imgs/nsmc-classification.png)

빠른 실험을 위해 1%의 데이터셋만 사용합니다. 본 데이터셋 기준으로는 총 1500건의 훈련 데이터를 사용합니다.

In [ ]:
# load dataset
train_dataset, test_dataset = load_dataset(dataset_name, split=['train[:1%]', 'test[:1%]'])

# num_samples_for_debug = 200
# train_dataset = train_dataset.shuffle(seed=42).select(range(num_samples_for_debug))
# test_dataset = test_dataset.shuffle(seed=42).select(range(num_samples_for_debug))

logger.info(f" loaded train_dataset length is: {len(train_dataset)}")
logger.info(f" loaded test_dataset length is: {len(test_dataset)}")
logger.info(train_dataset[0])

In [ ]:
train_dir = 'datasets/train'
test_dir = 'datasets/test'
!rm -rf {train_dir} {test_dir}

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True) 

if not os.listdir(train_dir):
    train_dataset.save_to_disk(train_dir)
if not os.listdir(test_dir):
    test_dataset.save_to_disk(test_dir)

# from datasets import load_from_disk
# train_dataset = load_from_disk(train_dir)
# test_dataset = load_from_disk(test_dir)

### Tokenization

자연어 처리 모델을 훈련하려면, 토큰화(Tokenization)를 통해 말뭉치(corpus; 자연어 처리를 위한 대량의 텍스트 데이터)를 토큰 시퀀스로 나누는 과정이 필요합니다. BERT 이전의 자연어 처리 모델은 주로 도메인 전문가들이 직접 토큰화해놓은 토크아니저(Mecab, Kkma 등)들을 사용했지만, BERT를 훈련하기 위한 토크나이저는 도메인 지식 필요 없이 말뭉치에서 자주 등장하는 서브워드(subword)를 토큰화합니다. GPT 기반 모델은 BPE(Byte-pair Encoding)라는 통계적 기법을 사용하며, BERT 및 ELECTRA 기반 모델은 BPE와 유사한 Wordpiece를 토크나이저로 사용합니다.

In [ ]:
# download tokenizer
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_id)

# tokenizer helper function
def tokenize(batch):
    return tokenizer(batch['document'], padding='max_length', max_length=128, truncation=True)

# tokenize dataset
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# set format for pytorch
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

### Argument parser

In [ ]:
def parser_args(train_notebook=False):
    parser = argparse.ArgumentParser()

    # Default Setting
    parser.add_argument("--epochs", type=int, default=5)
    parser.add_argument("--seed", type=int, default=42)
    parser.add_argument("--train_batch_size", type=int, default=32)
    parser.add_argument("--eval_batch_size", type=int, default=128)
    parser.add_argument("--warmup_steps", type=int, default=0)
    parser.add_argument("--learning_rate", type=str, default=5e-5)
    parser.add_argument("--disable_tqdm", type=bool, default=False)
    parser.add_argument("--fp16", type=bool, default=True)
    parser.add_argument("--tokenizer_id", type=str, default='monologg/koelectra-small-v3-discriminator')
    parser.add_argument("--model_id", type=str, default='monologg/koelectra-small-v3-discriminator')    

    # SageMaker Container environment
    parser.add_argument("--output_data_dir", type=str, default=os.environ["SM_OUTPUT_DATA_DIR"])
    parser.add_argument("--model_dir", type=str, default=os.environ["SM_MODEL_DIR"])
    parser.add_argument("--n_gpus", type=str, default=os.environ["SM_NUM_GPUS"])
    parser.add_argument("--training_dir", type=str, default=os.environ["SM_CHANNEL_TRAIN"])
    parser.add_argument("--test_dir", type=str, default=os.environ["SM_CHANNEL_TEST"])
    parser.add_argument('--chkpt_dir', type=str, default='/opt/ml/checkpoints')     

    if train_notebook:
        args = parser.parse_args([])
    else:
        args = parser.parse_args()
    return args

### Load Arguments

주피터 노트북에서 곧바로 실행할 수 있도록 설정값들을 로드합니다. 물론 노트북 환경이 아닌 커맨드라인에서도 아래 커맨드로 훈련 스크립트를 실행할 수 있습니다.
```shell
cd scripts & python3 train.py
```

In [ ]:
chkpt_dir = 'chkpt'
model_dir = 'model'
output_data_dir = 'data'    
!rm -rf {chkpt_dir} {model_dir} {output_data_dir} 

if os.environ.get('SM_CURRENT_HOST') is None:
    is_sm_container = False

    #src_dir = '/'.join(os.getcwd().split('/')[:-1])
    src_dir = os.getcwd()
    os.environ['SM_MODEL_DIR'] = f'{src_dir}/{model_dir}'
    os.environ['SM_OUTPUT_DATA_DIR'] = f'{src_dir}/{output_data_dir}'
    os.environ['SM_NUM_GPUS'] = str(1)
    os.environ['SM_CHANNEL_TRAIN'] = f'{src_dir}/{train_dir}'
    os.environ['SM_CHANNEL_TEST'] = f'{src_dir}/{test_dir}'

args = parser_args(train_notebook=True) 
args.chkpt_dir = chkpt_dir
logger.info("***** Arguments *****")
logger.info(''.join(f'{k}={v}\n' for k, v in vars(args).items()))

os.makedirs(args.chkpt_dir, exist_ok=True) 
os.makedirs(args.model_dir, exist_ok=True)
os.makedirs(args.output_data_dir, exist_ok=True) 

### Load pre-trained model

In [ ]:
# Prepare model labels - useful in inference API
labels = train_dataset.features["label"].names
num_labels = len(labels)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

# Set seed before initializing model
set_seed(args.seed)
    
# Download pytorch model
model = ElectraForSequenceClassification.from_pretrained(
    model_id, num_labels=num_labels, label2id=label2id, id2label=id2label
)

<br>

## 2. Hugging Face Fine-tuning
---

먼저, `TrainingArguments` 클래스를 인스턴스화화합니다. 이 클래스로 훈련에 필요한 다양한 하이퍼파라메터를 지정할 수 있습니다.

In [ ]:
# define training args
training_args = TrainingArguments(
    output_dir=args.chkpt_dir,
    overwrite_output_dir=True if get_last_checkpoint(args.chkpt_dir) is not None else False,
    num_train_epochs=args.epochs,
    per_device_train_batch_size=args.train_batch_size,
    per_device_eval_batch_size=args.eval_batch_size,
    warmup_steps=args.warmup_steps,
    fp16=args.fp16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    disable_tqdm=args.disable_tqdm,
    logging_dir=f"{args.output_data_dir}/logs",
    learning_rate=float(args.learning_rate),
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

매 에폭(epoch)마다 검증 데이터셋으로 정확도(accuracy), 정밀도(precision), 재현율(recall), F1 스코어를 계산하기 위한 함수를 정의합니다.

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# compute metrics function for binary classification
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

훈련을 수행하기 위한 `Trainer` 클래스를 인스턴스화화합니다.

In [ ]:
# create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics    
)

### Start Training

훈련을 수행합니다. 딥러닝 기반 자연어 처리 모델 훈련에는 GPU가 필수이며, 본격적인 훈련을 위해서는 멀티 GPU 및 분산 훈련을 권장합니다.
만약 멀티 GPU가 장착되어 있다면 `Trainer`에서 `총 배치 크기 = 배치 크기 x 멀티 GPU`로 지정한 다음 데이터 병렬화를 자동으로 수행합니다. 

In [ ]:
%%time
# train model
if get_last_checkpoint(args.chkpt_dir) is not None:
    logger.info("***** Continue Training *****")
    last_checkpoint = get_last_checkpoint(args.chkpt_dir)
    trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    trainer.train()

<br>

## 3. Evaluation and Prediction
---

### Evaluation
검증 셋에 대해 평가를 수행합니다.

In [ ]:
eval_result = trainer.evaluate(test_dataset)

In [ ]:
# writes eval result to file which can be accessed later in s3 ouput
with open(os.path.join(args.output_data_dir, "eval_results.txt"), "w") as writer:
    print(f"***** Evaluation results *****")
    for key, value in sorted(eval_result.items()):
        writer.write(f"{key} = {value}\n")
        logger.info(f"{key} = {value}\n")

### Prediction

정답 레이블과 예측 레이블의 결과를 혼동 행렬(confusion matrix)로 비교해 봅니다.

In [ ]:
import numpy as np
results = trainer.predict(test_dataset)
y_true = results.label_ids
y_pred = np.argmax(results.predictions, axis=1)

In [ ]:
def plot_confusion_matrix(cm, target_names=None, cmap=None, normalize=True, labels=True, title='Confusion matrix'):
    import itertools
    import matplotlib.pyplot as plt
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    
    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names)
        plt.yticks(tick_marks, target_names)
    
    if labels:
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            if normalize:
                plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                         horizontalalignment="center",
                         color="white" if cm[i, j] > thresh else "black")
            else:
                plt.text(j, i, "{:,}".format(cm[i, j]),
                         horizontalalignment="center",
                         color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

cf = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(cf, normalize=False)

In [ ]:
# Remove checkpoints
import shutil
dir_list = os.listdir(args.chkpt_dir)
for d in dir_list:
    shutil.rmtree(os.path.join(args.chkpt_dir, d))
    
trainer.save_model(args.model_dir)